# Sparse Matrix Application: PageRank Algorithm 

Welcome to the second homework for the high-performance computation of sparse matrices! In this work, you'll try to mesure the importance of webpages through a power iteration method.

By the end of this homework, you are expected to be able to:

- Construct sparse adjacency matrices starting from a given directed graph of webpages
- Reduce constructed adjacency matrix to be stochastic
- Solve the PageRank problem through power iteration method whose kernel is Sparse Matrix Vector operation (SpMV)
- Understand the impact of dampling factor

## Table of Contents
- [1 - Packages](#1)
- [2 - Problem Statement](#2)
- [3 - Construct Adjacency Matrix](#3)
- [4 - Reduce Adjacency Matrix to be Stochastic](#4)
- [5 - A simple PageRank](#5)
- [6 - PageRank with Damping](#6)

<a name='1'></a>
## 1 - Packages

In [ ]:
using LinearAlgebra
using SparseArrays
using Printf
using Random
using BenchmarkTools
import Plots

<a name='2'></a>
## 2 - Problem Statement

PageRank (PR) is an algorithm used by Google Search to rank web pages in their search engine results, which is a way of measuring the importance of website pages. 

According to Wikipedia:
> PageRank works by counting the number and quality of links to a page to determine a rough estimate of how important the website is. The underlying assumption is that more important websites are likely to receive more links from other websites. [[1]](https://en.wikipedia.org/wiki/PageRank)

PageRank is introduced by the co-founder of Google [Larray Page](https://en.wikipedia.org/wiki/Larry_Page). It is named after both the term of "web page" and the last name of Larray Page. PageRank is the first and one of the best known algorithm that was used by Google.

The figure below is a simple example of webgraph which shows the relations of 5 different webpages as a directed graph:
- Nodes: webpages
- Edges: hyperlinks
- Weights: vote from one link to another

<center>
<img src="../figs/simplegraph.png" alt="centered image">
</center>
<caption><center><font color='purple'><b>Figure 1</b>: a simple Webgraph </font></center></caption>

Not all web pages are equally "important". For example:

- https://www.bbc.com (BBC)

vs

- https://brunowu.github.io (My personal webpage)

There is a large diversity in the web-graph node connectivity, and the idea of PageRank is to rank pages by their link structure (webgraph). 

Roughly saying, the idea behand is
> Links as votes

Page is more important if it has more in-links. And the votes for all links in the webpage can be determined with a recursive step.

A "rank score" $r_j$ of a webpage $j$
$$r_j = \sum_{i\rightarrow j}\frac{r_i}{d_i} \tag{1}$$

in which $i$ represents the indices of all webpages which are linked to the webpage $j$, and $d_i$ refers to the degree of out-links of the webpage $i$. 

In the next section, we will at first construct a matrix describing the webgraph.

<a name='3'></a>
## 3 - Construct Adjacency Matrix

An adjacency matrix is a square matrix used to represent a finite graph. The elements of the matrix indicate whether pairs of vertices are adjacent or not in the graph. For a simple graph, the adjacency matrix is a (0,1)-matrix with its elements either being 1 or 0. 

The objective of this section is to construct this simple adjacency matrix for the webgraph given as below:

<center>
<img src="../figs/graph2.png" alt="centered image">
</center>
<caption><center><font color='purple'><b>Figure 2</b>: Webgraph 1 </font></center></caption>

### Initialization

In [ ]:
# The number of nodes of the webgraph above is:
N = 4
# Create a empty Sparse Matrix object in Julia of size N*N
A₁ = spzeros(Float64, N, N)
@info A₁

### Construction
For a adjacency matrix $M$, its element $M_{ij}$ refers the connection of hyperlink from the node $j$ to the node $i$. If $M_{ij}=0$, it means that there is no connection from the node $j$ to $i$, and if $M_{ij}=1$, it refers to an existence of hyperlink from the node $j$ to $i$.

In [ ]:
#fill the non-zeros part following the graph
A₁[1,3]=1 ### hyperlink 3->1
A₁[1,4]=1 ### hyperlink 4->1
A₁[2,1]=1 ### hyperlink 1->2
A₁[3,1]=1 ### hyperlink 1->3
A₁[3,2]=1 ### hyperlink 2->3
A₁[3,4]=1 ### hyperlink 4->3
A₁[4,1]=1 ### hyperlink 1->4
A₁[4,2]=1 ### hyperlink 2->4
### All the other elements of A₁ are 0, as default.

### Display A₁ in a sparse matrix format
display(A₁)

### Display A₁ in a dense matrix format
display(Matrix(A₁))

@info "the number of non-zero elements is:" nnz(A₁)
@info "the sparsity is:" nnz(A₁)/(N*N)

<a name='4'></a>
## 4 - Reduce Adjacency Matrix to be Stochastic

In general, a stochastic matrix is a matrix having column sums equal to 1. Reducing an adjacency matrix to a stochastic adjecency matrix adds a constraint which forces the uniqueness of problem to be solved.

For $j$th column of a stochastic adjacency matrix, the sum of whose elements is 1, each element represents the probability of a hyperlink from the node $j$ to all the available webpages.

A simple function which reduce an adjacency matrix to be stochastic is given as below:

In [ ]:
function stochasticAdjMat(A)
    """
    Reduce a simple adjacency matrix to be stochastic.
    
    Arguments:
    A -- a sparse matrix

    Returns:
    A -- a sparse matrix overwritten by the reduced stochastic matrix
    """
    summation = sum(A, dims = 1)
    for i = 1:size(A, 1)
        if summation[i] != 0 
            A[:,i] = A[:,i] ./ summation[i] 
        end
    end
    return A
end

In [ ]:
A₁ = stochasticAdjMat(A₁)

### Display A₁ in a sparse matrix format
display(A₁)

### Display A₁ in a dense matrix format
display(Matrix(A₁))

<a name='5'></a>
## 5 - A simplified PageRank

PageRank can be computed either iteratively or algebraically. In this module, we only foucs on the iterative method.

The iterative method can be viewed as the power iteration method, and the iterative process can be summarized as:

1. At $t=0$, an initial probability distribution $V$ is randomly generated.
2. At each time step, the computation, as detailed above, yields
$$V_{t+1} = AV_{t} \tag{2}$$
    in which A is the generated schoatisc adjacency matrix.
    
3. The probability calculation is made for each page at a time point, then repeated for the next time point. The computation ends when for some small $\epsilon$ 
$$|V_{t+1}-V_{t}| < \epsilon$$
    i.e, when convergence is assumed.
    
The most important kernel of this solver is $V_{t+1} = AV_{t}$, the Sparse Matrix Vector product operation. 

We provides a simple implementation of this solver as follows:

In [ ]:
function simplepagerank(A; maxIter::Int=100, tol=0.001, verbosity=0)
    """
    A first implementation of PageRank based on power iteration method.
    
    Arguments:
    A -- a sparse matrix
    
    Optional:
    maxIter -- maximum iterative steps before the convergence criteria is satisfied, default value is 100
    tol -- the convergence criteria, default value is 0.001
    verbosity -- if print the convergence steps (yes, if it is > 0), default value is 0

    Returns:
    v -- a probability vector which indicates the importance of all nodes.
    """    
    # get size of matrix
    N = size(A, 1)
    # generate initial guess vector in random
    v = rand(N) 
    # 1-normalization of initial guess vector
    v = v ./ norm(v, 1)
    iter = 1
    while iter < maxIter
        last_v = v
        # SpMV
        v = A * v
        if verbosity > 0
            msg = "PageRank: step: $iter, ||last_v - v||₂: "
            msg *= @sprintf("%.12e\n", norm(last_v - v))
            printstyled(msg, bold=false, color=:green)
        end
        #check the convergence
        if norm(last_v - v) < tol
            if verbosity > 0
                msg = "PageRank converged at step: $iter with ||last_v - v||₂: "
                msg *= @sprintf("%.12e\n", norm(last_v - v))
                printstyled(msg, bold=false, color=:blue)
            end
            break
        end
        iter += 1
    end
    return v
end

### Solve $A_1$ with PageRank

In [ ]:
v₁=simplepagerank(A₁; maxIter=100, tol=1e-4, verbosity=1);

In [ ]:
display(v₁)
@info sum(v₁) 

Question:
> Which is the most and the least important webpage in Figure 2?

Answer:
> Write your answer here.

### Hands-on

This hands-on is to find the importance of webpages on the webgraph below. Its objective is to get familiar with the process we introduced in this section.

<center>
<img src="../figs/graph1.png" alt="centered image">
</center>
<caption><center><font color='purple'><b>Figure 3</b>: Webgraph 2 </font></center></caption>

In [ ]:
#ToDo: Start
# The number of nodes of the webgraph above is:
N = ?
# Create a empty Sparse Matrix object in Julia of size N*N
A₂ = spzeros(Float64, N, N)
@info A₂

#fill the non-zeros part following the graph
A₂[?,?]=?

### Display A₂ in a sparse matrix format
display(A₂)

### Display A₂ in a dense matrix format
display(Matrix(A₂))

@info "the number of non-zero elements is:" nnz(A₂)
@info "the sparsity is:" nnz(A₂)/(N*N)

#ToDo: End

In [ ]:
#ToDo: Start
#Reudce A₂ to be stochastic

#Put your code here

#ToDo: End

In [ ]:
#ToDo: Start
#Solve by simplified PageRank

#Put your code here

#ToDo: End

In [ ]:
display(v₂)
@info sum(v₂) 

Question:
> Analyze the importance of different webpages.

Answer:
> Write your answer here.

<a name='6'></a>
## 6 - PageRank with Damping

The simplified PageRank doesn't always work. 
First, let's try to solve the problem formed by the webgraph below.

<center>
<img src="../figs/damping1.png" alt="centered image">
</center>
<caption><center><font color='purple'><b>Figure 4</b>: Webgraph 3 </font></center></caption>

In [ ]:
# The number of nodes of the webgraph above is:
N = 6
# Create a empty Sparse Matrix object in Julia of size N*N
A₃ = spzeros(Float64, N, N)
@info A₃

#fill the non-zeros part following the graph
A₃[1,3]=1
A₃[3,6]=1
A₃[3,2]=1
A₃[2,4]=1
A₃[5,2]=1
A₃[5,4]=1
A₃[4,5]=1
### Display A₃ in a sparse matrix format
display(A₃)

### Display A₃ in a dense matrix format
display(Matrix(A₃))

@info "the number of non-zero elements is:" nnz(A₃)
@info "the sparsity is:" nnz(A₃)/(N*N)

#Reudce A₂ to be stochastic
A₃ = stochasticAdjMat(A₃)

#Solve by PageRank
v₃=simplepagerank(A₃; maxIter=100, tol=1e-4, verbosity=1);

In [ ]:
display(v₃)
@info sum(v₃) 

A quick conclusion:

> Obviously, the simplified PageRank FAILED!!! The sum of the elementals of $V_3$ is not $1$ anymore.

This is due to the "spider traps" of some webpages (4 and 5 in Figure 4), in which random walk gets "stuck" in a "trap". This trap absorbs the importance.

From a point of view of linear algebra, this matrix $A_3$ is singluar, which makes power iteration method fail to find its largest eigenvalue. This can be verified by printing the eigenvalues of $A_3$ as follows.

In [ ]:
#try to output the eigenvalues of A₃
@info eigen!(Matrix(A₃)).values

### Introduce a Damping factor
The Google's solution for this issue is introducing a damping factor $\beta$. 

So a "rank score" $r_j$ of a webpage $j$ with a damping factor is defined as:
$$r_j = \sum_{i\rightarrow j}\beta\frac{r_i}{d_i}+(1-\beta)\frac{1}{N} \tag{3}$$  in which $N$ is the node number in the webgraph.

In [ ]:
#try with different damping factor
damping = 0.75
#The new adjacency matrix with damping
Ad = damping * A₃ .+ (1-damping) / N
#Ad isn't singular anymore
@info eigen!(Matrix(Ad)).values

A quick conclusion:
> This $Ad$ is not singular anymore.

We give the implementation of PageRank with damping factor as follows:

In [ ]:
function pagerank(A; maxIter::Int = 100, damping = 0.85, tol=0.001, verbosity=0)
    
    """
    A first implementation of PageRank based on power iteration method.
    
    Arguments:
    A -- a sparse matrix
    
    Optional:
    maxIter -- maximum iterative steps before the convergence criteria is satisfied, default value is 100
    damping -- damping factor, default value is 0.85
    tol -- the convergence criteria, default value is 0.001
    verbosity -- if print the convergence steps (yes, if it is > 0), default value is 0

    Returns:
    v -- a probability vector which indicates the importance of all nodes.
    """  
    
    N = size(A, 1)
    v = rand(N) 
    v = v ./ norm(v, 1)
    iter = 1
    while iter < maxIter
        last_v = v
        #The first part of this operation is SpMV. The second
        #part is BLAS1 operation, which is cheap
        v = damping * A * v .+ ((1 - damping) / N) * sum(v)
        if verbosity > 0
            msg = "PageRank: step: $iter, ||last_v - v||₂: "
            msg *= @sprintf("%.12e\n", norm(last_v - v))
            printstyled(msg, bold=false, color=:green)
        end
        if norm(last_v - v) < tol
            if verbosity > 0
                msg = "PageRank converged at step: $iter with ||last_v - v||₂: "
                msg *= @sprintf("%.12e\n", norm(last_v - v))
                printstyled(msg, bold=false, color=:blue)
            end
            break
        end
        iter += 1
    end
    return v
end

In [ ]:
#ToDo: Start
#Compute the PageRank vector of A₃, considering the damping constant
#to be successively 0, 0.15,0.5, 0.85, and 1

#put your code here

#ToDo: End

Question:
> Which is the best damping factor among $0$, $0.15$, $0.5$, $0.85$ and $1$?

Answer:
> Write your answer here.

### Webgraph with dead ends

The simplified PageRank doesn't always work. 
First, let's try to solve the problem formed by the webgraph below.

<center>
<img src="../figs/damping2.png" alt="centered image">
</center>
<caption><center><font color='purple'><b>Figure 5</b>: Webgraph 4 </font></center></caption>

#### First try with the simplified PageRank

In [ ]:
N = 8
A₄ = spzeros(Float64,N,N)
A₄[1,2]=1
A₄[1,3]=1
A₄[2,4]=1
A₄[2,5]=1
A₄[3,6]=1
A₄[3,7]=1
A₄[3,8]=1
    
### Display A₄ in a sparse matrix format
display(A₄)

### Display A₄ in a dense matrix format
display(Matrix(A₄))

@info "the number of non-zero elements is:" nnz(A₄)
@info "the sparsity is:" nnz(A₄)/(N*N)

#Reudce A₄ to be stochastic
A₄ = stochasticAdjMat(A₄)

#Solve by PageRank
v₄=simplepagerank(A₄; maxIter=100, tol=1e-4, verbosity=1);

In [ ]:
@info sum(v₄)
display(v₄)

A quick conclusion:

> Once again, the simplified PageRank FAILED!!! 

This is due to the "dead ends" of some webpages (4, 5, 6, 7 and 8 in Figure 4), in which random walk has "nowhere to go". This causes the "leak" of the importance.

#### Try with the PageRank with Damping factor

In [ ]:
#ToDo: Start
#Try with damping factor 0.85

#put your code here.

#ToDo: End
@info sum(v₄)
display(v₄)

Question:
> Interpret your results in terms of the relationship between the number of incoming links that each node has and its rank.

Answer:
> Write your answer here. 